In [5]:
from IPython.display import display, Markdown, Latex

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-18 00:26:15--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  5.12MB/s    in 0.2s    

2024-10-18 00:26:16 (5.12 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
DATA = "data/input.txt"
with open(DATA) as f:
    text = f.read()

In [6]:
display(Markdown(text[:500]))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor

In [7]:
# Vocab Size - All unique characters
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


## Tokenization

In [10]:
# Tokenization - Mapping Characters to Integers and vice-versa
encodings = {ch: i for i, ch in enumerate(chars)}
decodings = {i: ch for i, ch in enumerate(chars)}

encode = lambda enc_str: [encodings[char] for char in enc_str]
decode = lambda dec_int: ''.join([decodings[i] for i in dec_int])

print(encode("Hi There!"))
print(decode(encode("Hi There!")))

[20, 47, 1, 32, 46, 43, 56, 43, 2]
Hi There!


#### Tokenizing entire text

In [11]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
# train-val split
split = int(0.9*len(data))
train_data = data[:split]
val_data = data[split:]

In [13]:
block_size = 8 # Batching the input data, dont want all at once
train_data[:block_size+1] # +1 because 9 tokens include 8 examples => a1 -> a2; a1,a2 -> a3; a1,a2,a3 -> a4

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
X = train_data[:block_size] # input to transformer
y = train_data[1:block_size+1] # target

for t in range(block_size):
    context = X[:t+1]
    target = y[t]
    print(f"Input: {context}\tTarget: {target}")

Input: tensor([18])	Target: 47
Input: tensor([18, 47])	Target: 56
Input: tensor([18, 47, 56])	Target: 57
Input: tensor([18, 47, 56, 57])	Target: 58
Input: tensor([18, 47, 56, 57, 58])	Target: 1
Input: tensor([18, 47, 56, 57, 58,  1])	Target: 15
Input: tensor([18, 47, 56, 57, 58,  1, 15])	Target: 47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47])	Target: 58


In [69]:
torch.manual_seed(40)
batch_size = 4 # number of contexts
block_size = 8 # maximum context length

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:block_size+i] for i in ix])
    y = torch.stack([data[i+1:block_size+i+1] for i in ix])
    return x, y
    
xb, yb = get_batch('train')
print(f'inputs: {xb.shape}\n{xb}')
print(f'targets: {yb.shape}\n{yb}')
print('-----------')
print('32 different examples')

inputs: torch.Size([4, 8])
tensor([[46, 47, 51,  8,  0,  0, 23, 21],
        [ 0, 13, 52, 42,  1, 52, 43, 60],
        [46,  5, 42,  1, 49, 47, 52, 45],
        [63, 53, 59,  8,  1, 24, 53, 56]])
targets: torch.Size([4, 8])
tensor([[47, 51,  8,  0,  0, 23, 21, 26],
        [13, 52, 42,  1, 52, 43, 60, 43],
        [ 5, 42,  1, 49, 47, 52, 45,  6],
        [53, 59,  8,  1, 24, 53, 56, 42]])
-----------
32 different examples


### Baseline - Bigram Model

In [77]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding(idx) # (B,T,C) (Batch, Time, Channel)
        
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        for _ in range(max_new_tokens):
            
            logits, loss = self(idx)
            # focus on the last time step
            logits = logits[:, -1, :] # (B, C)
            print(logits)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)
            print(probs)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.6710, grad_fn=<NllLossBackward0>)


The expected loss in this case (Negative Log Likelihood)

e(loss) = -(ln(1/vocab_size))

-(ln(1/65)) = 4.17
